### 循迹-训练模型

在本示例中，我们将训练神经网络从输入图像得到与目标相对应的一组x，y值。

我们将使用PyTorch深度学习框架来为路径跟随应用训练ResNet18神经网络体系结构模型。

In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np

### 下载并提取数据

开始之前，您应该上传在``data_collection.ipynb``中创建的``road_following_<Date&Time>.zip``到机器人上的notebook中。

> 如果您在收集数据的JetBot上进行培训，则可以跳过此操作！

然后，您应该通过调用以下命令来提取该数据集：

In [ ]:
!unzip -q road_following.zip

您应该在文件浏览器中看到一个名为dataset_all的文件夹。

### 创建DataSet实例

在这里，我们创建一个自定义``torch.utils.data.Dataset``数据集的实现，实现``__len__和__getitem__``功能。此类负责加载图像并从图像文件名中解析x，y值。因为我们实现了``torch.utils.data.Dataset``类，所以我们可以使用所有的Torch数据工具:)

我们将一些转换（例如颜色抖动）硬编码到我们的数据集中。我们将随机的水平翻转设为可选（如果您想遵循非对称路径，例如需要“保持右边”的道路）。如果您的机器人是否遵循某些约定并不重要，可以启用翻转功能以扩充数据集。

In [ ]:
def get_x(path):
    """Gets the x value from the image filename"""
    return (float(int(path[3:6])) - 50.0) / 50.0

def get_y(path):
    """Gets the y value from the image filename"""
    return (float(int(path[7:10])) - 50.0) / 50.0

class XYDataset(torch.utils.data.Dataset):
    
    def __init__(self, directory, random_hflips=False):#将这里的值设置为True后会对图片进行反转
        self.directory = directory
        self.random_hflips = random_hflips
        self.image_paths = glob.glob(os.path.join(self.directory, '*.jpg'))
        self.color_jitter = transforms.ColorJitter(0.3, 0.3, 0.3, 0.3)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        
        image = PIL.Image.open(image_path)
        x = float(get_x(os.path.basename(image_path)))
        y = float(get_y(os.path.basename(image_path)))
        
        if float(np.random.rand(1)) > 0.5:
            image = transforms.functional.hflip(image)
            x = -x
        
        image = self.color_jitter(image)
        image = transforms.functional.resize(image, (224, 224))
        image = transforms.functional.to_tensor(image)
        image = image.numpy()[::-1].copy()
        image = torch.from_numpy(image)
        image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        return image, torch.tensor([x, y]).float()
    
dataset = XYDataset('dataset_xy', random_hflips=False)

### 将数据集分为训练集和测试集

读取数据集后，我们将数据集分为训练集和测试集。在此示例中，我们分给训练集90%，分给测试集10%。测试集将用于验证我们训练的模型的准确性。

In [ ]:
test_percent = 0.1
num_test = int(test_percent * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_test, num_test])

### 创建数据加载器来批量加载数据

我们使用``DataLoader``类来批量加载数据，打乱数据并允许使用多个子进程。在此示例中，我们使用的批处理大小为64。批处理大小将基于GPU可用的内存，并且它会影响模型的准确性。

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

### 定义神经网络模型

我们使用PyTorch TorchVision上可用的ResNet-18模型。

在称为迁移学习的过程中，我们可以将预训练的模型（在数百万张图像上进行训练）重新用于可能需要更少数据的新任务。

有关ResNet-18的更多详细信息： https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py

有关Transfer Learning的更多详细信息：https://www.youtube.com/watch?v=yofjFQddwHE 

In [5]:
model = models.resnet18(pretrained=True)

ResNet模型具有512维输入``in_features``的完全连接（fc）的最终层，因此我们将训练回归的``out_features``为1维。

最后，我们迁移模型到GPU上执行

In [ ]:
model.fc = torch.nn.Linear(512, 2)
device = torch.device('cuda')
model = model.to(device)

# 训练回归：

如果误差减小，我们将训练50个epoch，并保存最佳模型。

In [ ]:
NUM_EPOCHS = 70
BEST_MODEL_PATH = 'best_steering_model_xy.pth'
best_loss = 1e9

optimizer = optim.Adam(model.parameters())

for epoch in range(NUM_EPOCHS):
    
    model.train()
    train_loss = 0.0
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        train_loss += float(loss)
        loss.backward()
        optimizer.step()
    train_loss /= len(train_loader)
    
    model.eval()
    test_loss = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        test_loss += float(loss)
    test_loss /= len(test_loader)
    
    print('%f, %f' % (train_loss, test_loss))
    if test_loss < best_loss:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_loss = test_loss

训练完模型后，它将生成``best_steering_model_xy.pth``文件。该文件可以在live demo的notebook中使用。

如果您在JetBot以外的其他机器上进行训练，则需要将其上传到JetBot的``road_following``文件夹中。